In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00


In [ ]:
!pip install sacrebleu

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [ ]:
import pandas as pd
import numpy as np
from datasets import load_metric
bleu_metric = load_metric("sacrebleu")
bleu_metric.add(
 prediction="the the the the the the", reference=["the cat is on the mat"])
results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
results["precisions"] = [np.round(p,2) for p in results["precisions"]]
pd.DataFrame.from_dict(results, orient="index",columns=["Value"])

/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


ImportError: To be able to use sacrebleu, you need to install the following dependency: sacrebleu.
Please install it using 'pip install sacrebleu' for instance.

In [ ]:
from datasets import load_dataset

data = load_dataset("cnn_dailymail",'3.0.0')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
data["train"].column_names

['article', 'highlights', 'id']

### Outputting the length of article and summaries

In [ ]:
article = data["train"][1]['article']
highlight = data["train"][1]['highlights']

In [ ]:
print(f"Article (excerpt of 500 characters, total length: {len(article)})\n")
print("Article", article[:500])
print("highlight", highlight)

Article (excerpt of 500 characters, total length: 4051)

Article Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most s
highlight Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


### Output of different models

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')
string = "The U.S. are a country. The U.N. is an organization."
sent_tokenize(string)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['The U.S. are a country.', 'The U.N. is an organization.']

In [ ]:
def baseline(text):
  return "\n".join(sent_tokenize(text)[:3])

In [ ]:
sample = data["train"][0]["article"][:2000]
baseline(sample)

'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him.\nDaniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties.\n"I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month.'

In [ ]:
summaries = {}

In [ ]:
from transformers import pipeline, set_seed

set_seed(42)
pipe = pipeline("text-generation","gpt2-xl",device=0)
gpt_query = sample + "\nTL;DR:\n"
pipe_out = pipe(gpt_query, max_length=512, clean_up_tokenization_spaces=True)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box off

In [ ]:
summaries["gpt2"] = pipe_out[0]['generated_text'][len(gpt_query):]

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
#del model
del pipe_out
#del trainer
import gc
gc.collect()
gc.collect()

0

In [ ]:
pipe = pipeline("summarization","t5-large")
pipe_out = pipe(sample)
summaries["T5"] =  "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

In [ ]:
summaries

{'gpt2': 'Daniel Radcliffe is 18 now, but he had no idea his money making career would last all this time - when he was 13 himself, and had already worked in various small roles before turning his attention to the screen. If you want to see where he came from, watch his interview with',
 'T5': "Harry Potter star Daniel Radcliffe turns 18 on monday .\nthe young actor says he has no plans to fritter his cash away .\ndetails of how he'll mark his landmark birthday are under wraps ."}

In [ ]:
del pipe

In [ ]:
pipe = pipeline("summarization","t5-large")
pipe_out = pipe(sample)
summaries["T5"] =  "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

In [ ]:
pipe = pipeline("summarization","facebook/bart-large-cnn")
pipe_out = pipe(sample)
summaries["Bart"] =  "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
pipe = pipeline("summarization",model="google/pegasus-cnn_dailymail")
pipe_out = pipe(sample)


config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

"Harry Potter star Daniel Radcliffe gains access to a reported £20 million fortune .<n>Young actor says he has no plans to fritter his cash away .<n>Radcliffe's earnings from the first five Potter films have been held in a trust fund ."

In [ ]:
summaries["Pegasus"] =  pipe_out[0]["summary_text"].replace(".<n>",".\n")

In [ ]:
summaries

{'gpt2': 'Daniel Radcliffe is 18 now, but he had no idea his money making career would last all this time - when he was 13 himself, and had already worked in various small roles before turning his attention to the screen. If you want to see where he came from, watch his interview with',
 'T5': "Harry Potter star Daniel Radcliffe turns 18 on monday .\nthe young actor says he has no plans to fritter his cash away .\ndetails of how he'll mark his landmark birthday are under wraps .",
 'Bart': 'Harry Potter star Daniel Radcliffe turns 18 on Monday.\nHe gains access to a reported £20 million ($41.1 million) fortune.\nRadcliffe says he has no plans to fritter his cash away on fast cars, drink and parties.\nHis earnings from the first five Potter films have been held in a trust fund.',
 'Pegasus': "Harry Potter star Daniel Radcliffe gains access to a reported £20 million fortune .\nYoung actor says he has no plans to fritter his cash away .\nRadcliffe's earnings from the first five Potter fil

In [ ]:
reference = data["train"][0]["highlights"]

In [ ]:
!pip install rouge_score

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
from datasets import load_metric
rouge_metric = load_metric("rouge")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
for model_name in summaries:
  rouge_metric.add(predictions=summaries[model_name],reference=reference)
  score = rouge_metric.compute()
  print(score)

{'rouge1': AggregateScore(low=Score(precision=0.19230769230769232, recall=0.2564102564102564, fmeasure=0.21978021978021978), mid=Score(precision=0.19230769230769232, recall=0.2564102564102564, fmeasure=0.21978021978021978), high=Score(precision=0.19230769230769232, recall=0.2564102564102564, fmeasure=0.21978021978021978)), 'rouge2': AggregateScore(low=Score(precision=0.0196078431372549, recall=0.02631578947368421, fmeasure=0.02247191011235955), mid=Score(precision=0.0196078431372549, recall=0.02631578947368421, fmeasure=0.02247191011235955), high=Score(precision=0.0196078431372549, recall=0.02631578947368421, fmeasure=0.02247191011235955)), 'rougeL': AggregateScore(low=Score(precision=0.1346153846153846, recall=0.1794871794871795, fmeasure=0.15384615384615383), mid=Score(precision=0.1346153846153846, recall=0.1794871794871795, fmeasure=0.15384615384615383), high=Score(precision=0.1346153846153846, recall=0.1794871794871795, fmeasure=0.15384615384615383)), 'rougeLsum': AggregateScore(lo

In [ ]:
test = data["test"].shuffle(seed=42).select(range(100))

In [ ]:
from tqdm import tqdm
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

def chunks(list_of_elem, batch_size):
  for i in range(0, len(list_of_elem), batch_size):
    yield list_of_elem[i : i + batch_size]


def evaluate(model, dataset,tokenizer,metric, column_text="article", reference_text="highlights",batch_size=16):

  article_batches = list(chunks(test["article"], batch_size))
  reference_batches = list(chunks(test["highlights"], batch_size))

  for article_batch, reference_batch in tqdm(zip(article_batches,reference_batches),total = len(article_batches)):

    tokenize_text = tokenizer(article_batch,max_length=1024, padding="max_length", truncation=True,return_tensors="pt")

    summaries = model.generate(input_ids=tokenize_text["input_ids"].to(device),
attention_mask=tokenize_text["attention_mask"].to(device),
length_penalty=0.8, num_beams=8, max_length=128)

    decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries]

    decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]

    metric.add_batch(predictions=decoded_summaries, references=reference_batch)

  return metric.compute()



In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
score = evaluate(model, test, tokenizer , rouge_metric, column_text="article", reference_text="highlights",batch_size=8)

rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn,score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=["pegasus"])

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
import gc
gc.collect()

450

In [ ]:
del model
del tokenizer